<a href="https://colab.research.google.com/github/martin-dj/casa0018/blob/main/Week7/anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

First set up the necessary Python imports and set up Tensor Board which provides a visual output of the training process.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Load the TensorBoard notebook extension
#%load_ext tensorboard
%reload_ext tensorboard


# Clear any logs from previous runs
%rm -rf ./logs/

# Create Data

Create sine function data. We’ll use the NumPy linspace to generate x values ranging between 0 and 200*Pi (100 cycles) and NumPy sine function to generate sine values to the corresponding x. We subdivide the range into 1000 data points. We also add a linear componenent to the sine data to generate a trend.

We then add some noise to the data. Finally, we visualize the data.

In [ ]:
x = np.linspace(0, 200*3.14159, 1001) # in radians, 100 cycles, 10 data points per cycle
#y = np.sin(x) + 0.002*x
y = np.sin(x)

# Plot our data
plt.figure(figsize=(12,8))
plt.plot(x,y)
plt.show()

y += 0.1*np.random.randn(*y.shape) #Add some noise to the data to make it more realistic

# Plot our data
plt.figure(figsize=(12,8))
plt.plot(x, y, 'b.')
plt.show()

# Define a dataframe using x and y values.
df = pd.DataFrame(data=y,index=x,columns=['Sine'])


# Split Data
Split the data into train and validate subsets. The train dataset is used for training the model and the validate data set is used to validate the model against unseen data as it undergoes training. Normally there is a third unseen data set used for testing the trained model. However, the time series used here is sufficiently predictable to test 'by eye'.

First, we’ll check the length of the data frame and use a fraction of the data to validate our model. Now if we multiply the length of the data frame with val_percent and round the value (as we are using for indexing purpose) we’ll get the index position i.e., val_index. Last, we’ll split the train and validation data using the val_index.

In [ ]:
val_pecent = 0.2
len(df)*val_pecent
val_point = np.round(len(df)*val_pecent)
val_index = int(len(df) - val_point)
train = df.iloc[:val_index]
val = df.iloc[val_index:]

plt.figure(figsize=(12,8))
plt.xlim(0, 200*3.142)

print(len(df)) # 601
print(len(train))
print(len(val))

plt.plot(train)
plt.plot(val)


# Normalise
We need to normalise the data in the range 0-1. We use a scaler to determine the max and min of the complete data set and then use the scaler to scale both the training a validation data set

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df)
MinMaxScaler(copy=True, feature_range=(0, 1))
scaled_train = scaler.transform(train)
scaled_val = scaler.transform(val)

# Time Series Generator
One problem we’ll face when using time series data is, we must transform the data into sequences of samples with input data and target data before feeding it into the model. We should select the length of the data sequence (window length) in such a way so that the model has an adequate amount of input data to generalize and predict i.e. in this situation we must feed the model at least with one cycle of sine wave values.

The model takes the previous 20 data points (one window) as input data and uses it to predict the next point, which is then compared to the actual target value for backpropagation and gradient descent.

This process is time-consuming and difficult if we perform this manually, hence we’ll make use of the Keras Timeseries Generator which transforms the data automatically and ready to train models without heavy lifting.

We can see that the length of the scaled_train is 801 and the length of the generator is 41(801–20) i.e. if we perform the tuple unpacking of the generator function using X,y as variables, X comprises the 20 data points (training window) and y contains the 21st data point which the model uses for the prediction target. i.e. X0 = values at timesteps 0-19 and y0 = value at timestep 20; X1 = values at timesteps 1-20 and y1 = value at timestep 21; ... X781 = values at timesteps 781-800 and y781 = value at timestep 801.

We also create a validation_generator that operates on the validation data set (scaled_val in the code). The validator_generator is not used to train the model. Instead, it is used after each epoch to validate how well the current model fits an **unseen** data set.

The batch size indicates how many sequences (windows) are seen by the network before the network weights are updated. A batch size of 1 means the weights are updated after every sequence (window) is input to the network. In this case a batch size of 10 is used, which speeds up training.

An epoch is one complete pass through the training data set.

In [ ]:
length = 20 # sequence length - the length of the training window
batch_size = 10

# Using timeseries_dataset_from_array to create a generator
generator = timeseries_dataset_from_array(data=scaled_train, targets=scaled_train, sequence_length=length, batch_size=batch_size)

validation_generator = timeseries_dataset_from_array(scaled_val, scaled_val, sequence_length=length, batch_size=batch_size)

print(len(scaled_train)) # 801
# the generator object has no length attribute
# it is possible to determine the length, but this is more involved
# the below code will demonstrate how to find the length if required
print(len(list(generator.as_numpy_iterator())))

# Create a model

The code to create a LSTM is similar to that for earlier NNs you have already seen.

The variable (n_features) defined stands for the number of features in the training data i.e., as we are dealing with univariate data we’ll only have one feature whereas if we are using multivariate data containing multiple features then we must specify the count of features in our data.

In [ ]:
n_features = 1

# Encoder
encoder = Sequential()
encoder.add(LSTM(length, return_sequences=True, input_shape=(length, n_features)))

# Decoder
decoder = Sequential()
decoder.add(LSTM(length, return_sequences=False, input_shape=(length, length))) # Pass the output shape of the encoder
decoder.add(Dense(1))

# Autoencoder
autoencoder = Sequential([encoder, decoder])
autoencoder.compile(loss='mse', optimizer='adam')
autoencoder.summary()

In [ ]:
autoencoder.fit(generator, epochs=20, validation_data=validation_generator)

# Comparing Forecasts Using The Validation Data

Now let's compare the forecasts of the LSTM model using our validation data set.

Again remembering this is not a true test as the validation set was used to validate our models.

Before we plot the results we reverse the scaling transformations.

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]

current_batch = first_eval_batch.reshape(1, length, n_features)


for i in range(len(val)):
  current_pred = autoencoder.predict(current_batch)[0]

  test_predictions.append(current_pred)

  # The issue was with the slicing of current_batch[:,1:,1:] which results in an
  # array with shape (1, 19, 0) along dimension 2. We need to keep the dimension 2
  # to be 1 in order to append the new prediction which has shape (1,1).
  # Instead of current_batch[:,1:,1:], we should use current_batch[:,1:,:] to select
  # all rows, columns from index 1 onwards, and all elements along the third dimension
  # This results in the correct shape (1, 19, 1) allowing the append to happen without error.
  current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis = 1) # Changed this line to fix the dimensional error

true_predictions = scaler.inverse_transform(test_predictions)
val['LSTM Predictions'] = true_predictions
val.plot(figsize=(12,8))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import matplotlib.pyplot as plt

# ==== Generate Sinusoidal Data ====
def generate_sinusoidal_data(samples=1000, time_steps_in=50, time_steps_out=20, frequency=0.1, noise=0.0):
    """Generates sinusoidal time series data for training"""
    X, y = [], []
    t = np.linspace(0, samples * frequency, samples + time_steps_in + time_steps_out)
    data = np.sin(2 * np.pi * t) + noise * np.random.randn(len(t))  # Sin wave with noise

    for i in range(samples):
        X.append(data[i : i + time_steps_in])  # Past values
        y.append(data[i + time_steps_in : i + time_steps_in + time_steps_out])  # Future values

    X = np.array(X).reshape(samples, time_steps_in, 1)  # Reshape for LSTM
    y = np.array(y).reshape(samples, time_steps_out, 1)  # Reshape for LSTM
    return X, y

# Generate Data
time_steps_in = 50  # Number of past time steps
time_steps_out = 20  # Number of future steps to predict
num_samples = 1000
X, y = generate_sinusoidal_data(num_samples, time_steps_in, time_steps_out)

# ==== Encoder-Decoder Model ====
latent_dim = 64  # LSTM units

# Encoder
encoder_inputs = Input(shape=(time_steps_in, 1))
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]  # Context vector

# Decoder
decoder_inputs = Input(shape=(time_steps_out, 1))  # Input: Zeros during training
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(1)  # Predict next step value
decoder_outputs = decoder_dense(decoder_outputs)

# Define model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="mse")
model.summary()

# ==== Train Model ====
decoder_input_data = np.zeros_like(y)  # Zero padding during training (teacher forcing)
model.fit([X, decoder_input_data], y, batch_size=16, epochs=30, validation_split=0.1)

# ==== Inference (Prediction) ====
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Function to make predictions
def predict_sequence(input_seq):
    """Generates a predicted sequence from the trained model"""
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, time_steps_out, 1))  # Start with zeros

    predictions = []

    for _ in range(time_steps_out):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        predictions.append(output_tokens[:, -1, 0])  # Get last timestep prediction
        target_seq[:, -1, 0] = output_tokens[:, -1, 0]  # Update input for next step

        states_value = [h, c]

    return np.array(predictions).flatten()

# ==== Example Prediction ====
test_input = X[0].reshape(1, time_steps_in, 1)  # Use the first training example
predicted_sequence = predict_sequence(test_input)

# ==== Plot Results ====
plt.figure(figsize=(10, 4))
plt.plot(range(time_steps_in), test_input.flatten(), label="Input (Past)")
plt.plot(range(time_steps_in, time_steps_in + time_steps_out), y[0].flatten(), label="True Future")
plt.plot(range(time_steps_in, time_steps_in + time_steps_out), predicted_sequence, label="Predicted Future", linestyle="dashed")
plt.legend()
plt.xlabel("Time Step")
plt.ylabel("Value")
plt.title("Sinusoidal Time Series Forecasting with LSTM Encoder-Decoder")
plt.show()
